## Image Generator

This notebook extracts addresses from given street addresses and generates rooftop images. These images are used to generate dataset

Authors: Shaurya Tripathi, Shiva Srivastava

In [ ]:

from geopy.geocoders import Nominatim
import pandas as pd
import geopandas as gpd
import numpy as np
import folium
from folium import Marker
import warnings 
import requests
from PIL import Image
import IPython.display as display
warnings.filterwarnings('ignore')

#### Functions to get co-ordinates and address extractor

In [ ]:
# Get coordinates function. Key is connected to a user
def get_coordinates(address):
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"
    params = {
        "address": address,
        "key": "Your key"
    }
    try:
        response = requests.get(base_url, params=params)
        data = response.json()
        
        if data["status"] == "OK":
            result = data["results"][0]
            location = result["geometry"]["location"]
            latitude = location["lat"]
            longitude = location["lng"]
            return latitude, longitude
        else:
            error_message = data["error_message"] if "error_message" in data else "Unknown error"
            print(f"Error: {error_message}")
            return None
    except requests.exceptions.RequestException as e:
        print(f"Request Exception: {e}")
        return None

In [ ]:
# Reverse geocoding
def get_formatted_addr(lat, lng):
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"
    params = {
        "latlng": str(lat) + "," + str(lng),
        "key": "Your key"
    }
    try:
        response = requests.get(base_url, params=params)
        data = response.json()
        
        if data["status"] == "OK":
            result = data["results"][0]
            addr = result["formatted_address"]
            location = result["geometry"]["location"]
            latitude = location["lat"]
            longitude = location["lng"]
            #print(addr)
            #print(location)
            #print(latitude)
            #print(longitude)
            return addr, latitude, longitude
        else:
            error_message = data["error_message"] if "error_message" in data else "Unknown error"
            print(f"Error: {error_message}")
            return None
    except requests.exceptions.RequestException as e:
        print(f"Request Exception: {e}")
        return None

In [ ]:
## Function get_static_map tp get the fixed size map
def get_static_map(latitude, longitude, filename, zoom=20, size="1024x1024", maptype="satellite", key="key"):
    base_url = "https://maps.googleapis.com/maps/api/staticmap"
    params = {
        "center": f"{latitude},{longitude}",
        "zoom": zoom,
        "size": size,
        "maptype": maptype,
        "key": key
    }
    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        image_data = response.content
        with open(filename, "wb") as f:
            f.write(image_data)
        #print("Satellite image saved as satellite_image.png")
    else:
        print("Failed to retrieve the satellite image.")

In [ ]:
def print_map(address, filename):

    latitude, longitude = get_coordinates(address)

    get_static_map(latitude, longitude, filename)

    image = Image.open(filename)

    # Display the image in Jupyter Notebook
    display.display(image)

In [ ]:
def create_address_csv(df, csv_name, loop_count):
    street_list = df['Address'].unique().tolist()
    num_streets = len(street_list)
    
    temp_list=[]
    # Loop through each street and get house address
    for i in range(num_streets):
        street = street_list[i]
        print(street)
        lat, long = get_coordinates(street)
        address, latitude, longitude = get_formatted_addr(lat, long)
        temp_list.append(get_formatted_addr(lat, long))
        #print(temp_list)
        
        for j in range(loop_count):
            latitude += 0.0001
            longitude += 0.0001
            temp_list.append(get_formatted_addr(latitude, longitude))
            

    # drop duplicates
    df_temp = pd.DataFrame(temp_list, columns=['Address', 'Latitude', 'Longitude'])
    df_temp = df_temp.drop_duplicates(subset=['Address'])
    df_temp.to_csv(csv_name)   

In [ ]:
def generate_images(csv, filepath, filename):
    df = pd.read_csv(csv)
    num = len(df)
    for i in range(num):
        lat = df.iloc[i]['Latitude']
        long = df.iloc[i]['Longitude']
        get_static_map(lat, long, filepath+filename+"_" + str(i) + ".png")
        #print(lat)
    print("Done Generating images.")


#### Make dataframes city street list csv


In [ ]:
# Creat a dataframe of Saratoga street addresses
# Created Saratoga.csv from https://geographic.org/streetview/usa/ca/saratoga.html
df_saratoga = pd.read_csv('../data/Saratoga.csv', delimiter="\,", header=0)
df_saratoga["Address"] = df_saratoga[["address2", "city", "state"]].apply(lambda x: ", ".join(x), axis=1)
df_saratoga.drop(['address1', 'address2', 'state', 'city'], axis=1, inplace=True)
df_saratoga.head()


In [ ]:
# Creat a dataframe of Fremont street addresses
df_fremont = pd.read_csv('../data/Fremont.csv', delimiter="\,", header=0)
df_fremont["Address"] = df_fremont[["address2", "city", "state"]].apply(lambda x: ", ".join(x), axis=1)
df_fremont.drop(['address1', 'address2', 'state', 'city'], axis=1, inplace=True)
df_fremont.head()

In [ ]:
# Creat a dataframe of Fremont street addresses
df_cupertino = pd.read_csv('../data/Cupertino.csv', delimiter="\,", header=0)
df_cupertino["Address"] = df_cupertino[["address2", "city", "state"]].apply(lambda x: ", ".join(x), axis=1)
df_cupertino.drop(['address1', 'address2', 'state', 'city'], axis=1, inplace=True)
df_cupertino.head()

In [ ]:
# Creat a dataframe of Fremont street addresses
# df_test = pd.read_csv('./test.csv', delimiter="\,", header=0)
# df_test["Address"] = df_test[["address2", "city", "state"]].apply(lambda x: ", ".join(x), axis=1)
# df_test.drop(['address1', 'address2', 'state', 'city'], axis=1, inplace=True)
# df_test.head()

#### Create csv files of home addresses, latitude and longitude

In [ ]:
create_address_csv(df_test, "test_home_addresses.csv", 2)

In [ ]:
generate_images("test_home_addresses.csv", "OUT_DIR", "temp")

In [ ]:
create_address_csv(df_saratoga, "saratoga_addresses.csv", 4)

In [ ]:
create_address_csv(df_fremont, "fremont_addresses.csv", 2)

In [ ]:
create_address_csv(df_cupertino, "cupertino_addresses.csv", 3)

In [ ]:
generate_images("saratoga_addresses.csv", "OUT_DIR", "Saratoga")

In [ ]:
generate_images("cupertino_addresses.csv", "OUT_DIR", "Cupertino")

In [ ]:
generate_images("fremont_addresses.csv", "OUT_DIR", "Fremont")